In [3]:
import requests
import json
import random, datetime
from faker import Faker

with open("./mock-insert/referenten.json") as file:
    referenten = json.load(file)

with open("./mock-insert/studiengaenge.json") as file:
    studiengaenge = json.load(file)

with open("./mock-insert/studenten.json") as file:
    studenten = json.load(file)

for ref in referenten:
    requests.post("http://localhost:8080/api/evaluator", json=ref)

for studg in studiengaenge:
    requests.post("http://localhost:8080/api/study-program", json=studg)

for stud in studenten:
    requests.post("http://localhost:8080/api/student", json=stud)

# create works by combining random referent, student, study program
# 30 refs, 50 studs, ~80 stupgs

def random_iso_8601_date(minm, maxm):
    start_date = datetime.date(2025, minm, 1)
    end_date = datetime.date(2025, maxm, 28)
    step = datetime.timedelta(days=1)
    dates = [ start_date ]

    while start_date != end_date:
        start_date += step
        dates.append(start_date)

    return random.choice(dates).isoformat()

def random_iso_8601_datetime(minm, maxm, st):
    start_date = datetime.datetime(2025, minm, 1)
    end_date = datetime.datetime(2025, maxm, 28)
    step = datetime.timedelta(days=1)
    dates = [ start_date ]

    while start_date != end_date:
        start_date += step
        dates.append(start_date)

    dt = random.choice(dates)
    dt = dt.replace(hour=st, minute=0)
    return dt.isoformat()

fake = Faker()

for _ in range(100):
    starttime = random.randint(8, 14)
    ref1 = random.randint(1, len(referenten))
    ref2 = random.randint(1, len(referenten))

    while ref1 == ref2:
        ref2 = random.randint(1, len(referenten))

    work = {
        "title": " ".join(fake.sentences()),
        "colloquium": random_iso_8601_datetime(11, 12, starttime),
        "colloquiumLocation": f"Room {random.randint(1, 400)}",
        "colloquiumDuration": "PT1H30M", 
        "presentationStart": f"{starttime}:00",
        "presentationEnd": f"{starttime}:30",
        "discussionStart": f"{starttime}:30",
        "discussionEnd": f"{starttime + 1}:00",
        "startDate": random_iso_8601_date(2, 6),
        "endDate": random_iso_8601_date(9, 10),
        "studentId": random.randint(1, len(studenten)),
        "studyProgramId": random.randint(1, len(studiengaenge)),
        "mainEvaluatorId": ref1,
        "mainEvaluatorWorkMark": random.randint(50, 100),
        "mainEvaluatorColloquiumMark": random.randint(50, 100),
        "secondEvaluatorId": ref2,
        "secondEvaluatorWorkMark": random.randint(50, 100),
        "secondEvaluatorColloquiumMark": random.randint(50, 100)
    }

    print(work)
    res = requests.post("http://localhost:8080/api/scientific-work", json=work)


{'title': 'Money move determine knowledge tonight voice. Yet cover officer military movement identify. Difficult against material line design relate thing.', 'colloquium': '2025-12-05T11:00:00', 'colloquiumLocation': 'Room 132', 'colloquiumDuration': 'PT1H30M', 'presentationStart': '11:00', 'presentationEnd': '11:30', 'discussionStart': '11:30', 'discussionEnd': '12:00', 'startDate': '2025-03-14', 'endDate': '2025-10-11', 'studentId': 44, 'studyProgramId': 61, 'mainEvaluatorId': 15, 'mainEvaluatorWorkMark': 91, 'mainEvaluatorColloquiumMark': 93, 'secondEvaluatorId': 25, 'secondEvaluatorWorkMark': 64, 'secondEvaluatorColloquiumMark': 64}
{'title': 'Now item debate debate keep manage fish. Moment face let social argue. Space boy fly teacher voice television picture keep.', 'colloquium': '2025-12-19T13:00:00', 'colloquiumLocation': 'Room 14', 'colloquiumDuration': 'PT1H30M', 'presentationStart': '13:00', 'presentationEnd': '13:30', 'discussionStart': '13:30', 'discussionEnd': '14:00', 'st